In [1]:
import pandas as pd
import spacy

spacy.prefer_gpu()

True

In [2]:
dados_treino = pd.read_csv("../introducao-word-embedding/treino.csv")
dados_treino.sample(5)

,title,text,date,category,subcategory,link
39056,Crítica: 'Riso Nervoso' faz rir com situações ...,Apesar da proposta de experimentar um formato ...,2015-12-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2015/10...
63062,"Com R$ 230 mil para vencedores, novo prêmio li...",Após meses de impasse em relação à realização ...,2015-09-06,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2015/06...
66851,"Líderes devem dar mais poder às máquinas, defe...",Quando a inteligência artificial é má notícia ...,2017-08-19,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...
54173,"Único a perder três finais de Copa América, Ma...",Mascherano passa por uma dor que pode ser desc...,2015-05-07,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/07/1...
42090,Professor disparou contra atiradores para prot...,"""Mártir"" e ""cavalheiro"" são alguns dos elogios...",2016-01-20,mundo,NaN,http://www1.folha.uol.com.br/mundo/2016/01/173...


In [3]:
#Rodas apenas uma vez
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 72.7 MB/s eta 0:00:00m eta 0:00:01:01:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [4]:
nlp = spacy.load("pt_core_news_sm")

In [5]:
texto = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(texto)

In [6]:
doc

Rio de Janeiro é uma cidade maravilhosa

In [7]:
type(doc[2])

spacy.tokens.token.Token

In [8]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])

In [9]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2:
        return  " ".join(tokens_validos)

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [10]:
texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [11]:
from time import time

t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 1000, 
                                                        #n_process = -1
                                                        )]

tf = time() - t0

print(tf/60)

0.37393805583318074


In [12]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [13]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [14]:
w2v_modelo

In [15]:
print(len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

90000
84466


In [16]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [17]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

2024-06-06 16:32:28,533 : - Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2024-06-06T16:32:28.533381', 'gensim': '4.3.0', 'python': '3.11.9 | packaged by conda-forge | (main, Apr 19 2024, 18:36:13) [GCC 12.3.0]', 'platform': 'Linux-6.8.0-35-generic-x86_64-with-glibc2.39', 'event': 'created'}
2024-06-06 16:32:28,533 : - collecting all words and their counts
2024-06-06 16:32:28,534 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-06-06 16:32:28,537 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2024-06-06 16:32:28,540 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2024-06-06 16:32:28,544 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2024-06-06 16:32:28,547 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2024-06-06 16:32:28,551 : - PROGRESS: at sentence #25000, processed 15

In [18]:
dir(w2v_modelo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 

In [19]:
w2v_modelo.corpus_count

84466

In [20]:
w2v_modelo.train(lista_lista_tokens, 
                 total_examples=w2v_modelo.corpus_count,
                 epochs = 30)

2024-06-06 16:32:28,711 : - Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2024-06-06T16:32:28.711682', 'gensim': '4.3.0', 'python': '3.11.9 | packaged by conda-forge | (main, Apr 19 2024, 18:36:13) [GCC 12.3.0]', 'platform': 'Linux-6.8.0-35-generic-x86_64-with-glibc2.39', 'event': 'train'}
2024-06-06 16:32:28,935 : - EPOCH 0: training on 540242 raw words (486037 effective words) took 0.2s, 2204221 effective words/s
2024-06-06 16:32:29,151 : - EPOCH 1: training on 540242 raw words (486138 effective words) took 0.2s, 2301347 effective words/s
2024-06-06 16:32:29,365 : - EPOCH 2: training on 540242 raw words (486165 effective words) took 0.2s, 2311997 effective words/s
2024-06-06 16:32:29,577 : - EPOCH 3: training on 540242 raw words (486203 effective words) took 0.2s, 2328941 effective words/s
2024-06-06 16:32:29,791 : - EPOCH 4: training on 540242 r

(14584209, 16207260)

In [21]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.5853783488273621),
 ('facebook', 0.5384712219238281),
 ('airbnb', 0.48048362135887146),
 ('waze', 0.46768882870674133),
 ('uber', 0.45991137623786926),
 ('amazon', 0.4587847888469696),
 ('yahoo', 0.4543995261192322),
 ('reputação', 0.44935259222984314),
 ('promotora', 0.4446452856063843),
 ('tesla', 0.4417654573917389)]

In [22]:
w2v_modelo.wv.most_similar("microsoft")

[('unilever', 0.5704851150512695),
 ('amazon', 0.5526418685913086),
 ('sky', 0.5364922881126404),
 ('tesla', 0.49938392639160156),
 ('buffett', 0.4972078800201416),
 ('braskem', 0.4934007227420807),
 ('walmart', 0.48912063241004944),
 ('syngenta', 0.48294442892074585),
 ('bingo', 0.4799862504005432),
 ('sony', 0.4760758876800537)]

In [23]:
w2v_modelo.wv.most_similar("barcelona")

[('bayern', 0.5606571435928345),
 ('barça', 0.556068480014801),
 ('madrid', 0.5485405921936035),
 ('united', 0.5452345013618469),
 ('lazio', 0.5346264243125916),
 ('juventus', 0.5250296592712402),
 ('munique', 0.5249300599098206),
 ('chelsea', 0.5179049968719482),
 ('psg', 0.5076916813850403),
 ('liverpool', 0.49374499917030334)]

In [24]:
w2v_modelo.wv.most_similar("messi")

[('suárez', 0.5900891423225403),
 ('chuteiras', 0.5271071791648865),
 ('cristiano', 0.5209771990776062),
 ('cavani', 0.5183922052383423),
 ('neymar', 0.5141710638999939),
 ('tevez', 0.5139928460121155),
 ('enrique', 0.4907070994377136),
 ('benzema', 0.48883572220802307),
 ('barça', 0.4762296676635742),
 ('psg', 0.4673621952533722)]

In [25]:
w2v_modelo.wv.most_similar("gm")

[('volks', 0.6724461317062378),
 ('embraer', 0.6514822840690613),
 ('chrysler', 0.6378532648086548),
 ('honda', 0.6226741075515747),
 ('braskem', 0.6017970442771912),
 ('csn', 0.5902283191680908),
 ('toyota', 0.5754856467247009),
 ('tesla', 0.5674526691436768),
 ('fiat', 0.5630203485488892),
 ('metalúrgicos', 0.5601111054420471)]

In [26]:
#Treinamento do modelo Skip-Gram
w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens, 
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs = 30)

2024-06-06 16:32:35,197 : - Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2024-06-06T16:32:35.197448', 'gensim': '4.3.0', 'python': '3.11.9 | packaged by conda-forge | (main, Apr 19 2024, 18:36:13) [GCC 12.3.0]', 'platform': 'Linux-6.8.0-35-generic-x86_64-with-glibc2.39', 'event': 'created'}
2024-06-06 16:32:35,200 : - collecting all words and their counts
2024-06-06 16:32:35,201 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-06-06 16:32:35,210 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2024-06-06 16:32:35,218 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2024-06-06 16:32:35,225 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2024-06-06 16:32:35,237 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2024-06-06 16:32:35,245 : - PROGRESS: at sentence #25000, processed 15

(14584512, 16207260)

In [27]:
w2v_modelo_sg.wv.most_similar("google")

[('reguladores', 0.4501875042915344),
 ('android', 0.4084903299808502),
 ('waze', 0.38049954175949097),
 ('concorda', 0.3795197010040283),
 ('facebook', 0.3745160698890686),
 ('buffett', 0.3742385506629944),
 ('verizon', 0.36863839626312256),
 ('walmart', 0.36834385991096497),
 ('toshiba', 0.36084863543510437),
 ('yahoo', 0.35604071617126465)]

In [28]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.5853783488273621),
 ('facebook', 0.5384712219238281),
 ('airbnb', 0.48048362135887146),
 ('waze', 0.46768882870674133),
 ('uber', 0.45991137623786926),
 ('amazon', 0.4587847888469696),
 ('yahoo', 0.4543995261192322),
 ('reputação', 0.44935259222984314),
 ('promotora', 0.4446452856063843),
 ('tesla', 0.4417654573917389)]

In [29]:
w2v_modelo_sg.wv.most_similar("gm")

[('metalúrgicos', 0.592609167098999),
 ('motors', 0.53378826379776),
 ('audi', 0.48546361923217773),
 ('airbags', 0.4805930554866791),
 ('fiat', 0.4751640558242798),
 ('bmw', 0.4749111831188202),
 ('honda', 0.47376903891563416),
 ('chrysler', 0.4653456509113312),
 ('airbag', 0.45837751030921936),
 ('cubatão', 0.4563847780227661)]

In [30]:
w2v_modelo.wv.most_similar("gm")

[('volks', 0.6724461317062378),
 ('embraer', 0.6514822840690613),
 ('chrysler', 0.6378532648086548),
 ('honda', 0.6226741075515747),
 ('braskem', 0.6017970442771912),
 ('csn', 0.5902283191680908),
 ('toyota', 0.5754856467247009),
 ('tesla', 0.5674526691436768),
 ('fiat', 0.5630203485488892),
 ('metalúrgicos', 0.5601111054420471)]

In [31]:
w2v_modelo.wv.save_word2vec_format("modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("modelo_skipgram.txt", binary=False)

2024-06-06 16:32:52,453 : - storing 12924x300 projection weights into modelo_cbow.txt
2024-06-06 16:32:53,374 : - storing 12924x300 projection weights into modelo_skipgram.txt


In [32]:
w2v_modelo.wv.save_word2vec_format("rl_cbow.pkl", binary=True)
w2v_modelo_sg.wv.save_word2vec_format("rl_sg.pkl", binary=True)

2024-06-06 16:32:54,180 : - storing 12924x300 projection weights into rl_cbow.pkl
2024-06-06 16:32:54,206 : - storing 12924x300 projection weights into rl_sg.pkl
